In [31]:
# New technology used
#  %%capture
# SentencePiece as a supplement to NLTK

In [4]:
# Create an application that will allow users to input symptoms, which can then be paired with possible diseases and offer treatements
# Will use gradio to build this app



!pip install gradio
!pip intall nltk
!pip install transformers
!pip install torch
!pip install sentencepiece
!pip install tensorflow
!pip install tensorflow_hub
!pip install tensorflow_text
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn
!pip install sqlite3
!pip install extract_named_entities_nltk




ERROR: unknown command "intall" - maybe you meant "install"



ERROR: Could not find a version that satisfies the requirement tensorflow_text (from versions: none)
ERROR: No matching distribution found for tensorflow_text


ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3
ERROR: Could not find a version that satisfies the requirement extract_named_entities_nltk (from versions: none)
ERROR: No matching distribution found for extract_named_entities_nltk


In [5]:
# Imports needed for this application
import gradio as gr
import torch
import sentencepiece
import tensorflow
import tensorflow_hub
import tensorflow_text
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3




ImportError: Traceback (most recent call last):
  File "c:\Users\jclar\anaconda3\envs\dev\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 70, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [34]:
from google.colab import files
uploaded = files.upload()

Saving symbipredict.csv to symbipredict (4).csv


In [35]:
#  Read the .csv using pandas
import pandas as pd

# Load the data
disease_data = pd.read_csv('symbipredict.csv')

# Display the first few rows of the dataset
print(disease_data.head())

            Disease  Symptom_1             Symptom_2             Symptom_3  \
0  Fungal Infection    itching             skin_rash  nodal_skin_eruptions   
1  Fungal Infection  skin_rash  nodal_skin_eruptions   dischromic _patches   
2  Fungal Infection    itching  nodal_skin_eruptions   dischromic _patches   
3  Fungal Infection    itching             skin_rash   dischromic _patches   
4  Fungal Infection    itching             skin_rash  nodal_skin_eruptions   

             Symptom_4 Symptom_5 Symptom_6 Symptom_7 Symptom_8 Symptom_9  \
0  dischromic _patches       NaN       NaN       NaN       NaN       NaN   
1                  NaN       NaN       NaN       NaN       NaN       NaN   
2                  NaN       NaN       NaN       NaN       NaN       NaN   
3                  NaN       NaN       NaN       NaN       NaN       NaN   
4                  NaN       NaN       NaN       NaN       NaN       NaN   

  Symptom_1.1 Symptom_11 Symptom_12 Symptom_13 Symptom_14 Symptom_15  \
0 

In [36]:
# After loading the data, it is necessary to combine the symptom_columns into a single column
symptom_columns = [col for col in disease_data.columns if col != 'Disease']
disease_data['Processed_Symptoms'] = disease_data[symptom_columns].apply(lambda x: ' '.join(x.astype(str)), axis=1)

In the section below, we will use the patient's diagnosis to suggest possible treatments.
First, we have to load the treatment database that we have selected (name it here).  Then, we will run a process to link the disease to a Treatment  using Pandas Dataframe (for CSV)

In [37]:
# Import necessary libraries starting with nltk
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import gradio as gr
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('maxent_ne_chunker')
nltk.download('words')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [38]:
# Define the prepocessing of the data
def preprocess_symptoms(symptom_text):
    tokens = word_tokenize(symptom_text.lower())
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [w for w in tokens if not w in stop_words and w.isalnum()]
    return ' '.join(filtered_tokens)

# Example synonym dictionary (this can be expanded)
synonym_dict = {
    'fever': ['fever', 'pyrexia'],
    'headache': ['headache', 'migraine', 'cephalalgia'],
    'nausea': ['nausea', 'queasiness', 'sickness'],
    'vomiting': ['vomiting', 'throwing up', 'emesis'],
    'sore throat': ['sore throat', 'pharyngitis', 'throat pain']
}

def expand_keywords(keywords):
    expanded_keywords = set()
    for keyword in keywords:
        if keyword in synonym_dict:
            expanded_keywords.update(synonym_dict[keyword])
        else:
            expanded_keywords.add(keyword)
    return list(expanded_keywords)

def extract_keywords(patient_feedback):
    tokens = word_tokenize(patient_feedback.lower())
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [w for w in tokens if not w in stop_words and w.isalnum()]
    pos_tags = nltk.pos_tag(filtered_tokens)
    keywords = [word for word, pos in pos_tags if pos.startswith('NN') or pos.startswith('JJ') or pos.startswith('VB')]
    return keywords






In [39]:
# Here the application will read the symptoms and suggest some diagnosis
# We will use gradio to build our app
def suggest_diagnosis_tfidf(patient_feedback):
    keywords = extract_keywords(patient_feedback)
    expanded_keywords = expand_keywords(keywords)
    print("Expanded Keywords:", expanded_keywords)

    processed_feedback = ' '.join(expanded_keywords)
    vectorizer = TfidfVectorizer()
    symptom_matrix = vectorizer.fit_transform(disease_data['Processed_Symptoms'])
    feedback_vector = vectorizer.transform([processed_feedback])
    similarities = cosine_similarity(feedback_vector, symptom_matrix)
    sorted_indices = similarities.argsort()[0][::-1]

    possible_diagnoses = []
    added_diseases = set()  # To track added diagnoses and avoid duplicates
    for idx in sorted_indices:
        disease_name = disease_data.loc[idx, 'Disease']
        if disease_name not in added_diseases:
            possible_diagnoses.append(disease_name)
            added_diseases.add(disease_name)

    if not possible_diagnoses:
        possible_diagnoses = ["Unable to determine a diagnosis based on the provided information."]

    return possible_diagnoses[:5]  # Return top 5 possible diagnoses

# Example usage
feedback = "fever, headache, nausea, vomiting, sore throat"
diagnosis = suggest_diagnosis_tfidf(feedback)
print("Suggested Diagnosis:", diagnosis)

# Gradio Interface
iface = gr.Interface(
    fn=suggest_diagnosis_tfidf,
    inputs=gr.Textbox(lines=5, placeholder="Describe your symptoms..."),
    outputs="text",
    title="Symptom Checker",
    description="Enter your symptoms, and we'll suggest possible diagnoses.",
)

iface.launch()


Expanded Keywords: ['queasiness', 'headache', 'sickness', 'sore', 'emesis', 'cephalalgia', 'migraine', 'nausea', 'vomiting', 'throwing up', 'throat']
Suggested Diagnosis: ['Dengue', 'Typhoid', 'Malaria', 'Hypoglycemia', 'Vertigo']
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ec0cfa1ff8e1a23ad8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Now that we have established the symptom to diagnosis part of the application, our next piece is to make some suggested remedies for the suggested diagnosis.

In [40]:
#  Load and Access Treatment Database
treatment_database = pd.read_csv('treatment_database.csv')
treatment_database.head()

FileNotFoundError: [Errno 2] No such file or directory: 'treatment_database.csv'

In [ ]:
# File import
import sqlite3
        conn = sqlite3.connect('treatment_database.db')
        cursor = conn.cursor()
        cursor.

In [ ]:
# Link the disease to a possible treatment
def get_treatments(diagnosed_disease):
        treatments = treatment_data[treatment_data['disease'] == diagnosed_disease]['treatment'].tolist()
        if treatments:
            return treatments
        else:
            return "No treatments found for this disease."